## Characterising Score Distributions in Dice Games

Isaksen, Aaron, et al. "Characterising score distributions in dice games." Game and Puzzle Design 2.1 (2016): 14.

[PDF from Julian Togelius's website.](http://julian.togelius.com/Isaksen2016Characterising.pdf)

We reproduce all tables presented in this paper.
Probably the most interesting calculation is 8.1 Rolling Sorted, Rerolling Tied Dice.

In [1]:
import piplite
await piplite.install("icepool")

import icepool
from math import sqrt

import time

start_ns = time.perf_counter_ns()

In [2]:
# 2. Metrics for Dice Games

def win_bias(die):
    return 100.0 * ((die > 0).mean() - (die < 0).mean())

def tie_percentage(die):
    return 100.0 * (die == 0).mean()

def closeness(die):
    return 1.0 / sqrt(die.sub(lambda x: x * x).mean())

def print_header():
    print('|     Game     | win bias | tie % | closeness |')
    print('|:------------:|---------:|------:|----------:|')

def print_row(game, die):
    print(f'| {game:12} |   {win_bias(die):6.2f} | {tie_percentage(die):5.2f} |     {closeness(die):5.3f} |')

In [3]:
# 3.1 Sorting Dice, With Ties

class SortingDiceWithTies(icepool.OutcomeCountEvaluator):
    def next_state(self, state, outcome, a, b):
        net_score, advantage = state or (0, 0)
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        # We pair them off with newly-rolled dice of the disadvantaged side.
        if advantage > 0:
            net_score += min(b, advantage)
        elif advantage < 0:
            net_score -= min(a, -advantage)
        advantage += a - b
        return net_score, advantage
    
    def final_outcome(self, final_state, *_):
        # Take only the score.
        return final_state[0]
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1

sorting_dice_with_ties = SortingDiceWithTies()

print_header()
for die_size in [2, 4, 6, 8, 10]:
    pool = icepool.d(die_size).pool(5)
    result = sorting_dice_with_ties.evaluate(pool, pool)
    print_row(f'5d{die_size}', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d2          |     0.00 | 24.61 |     0.632 |
| 5d4          |     0.00 | 11.97 |     0.384 |
| 5d6          |     0.00 |  9.91 |     0.340 |
| 5d8          |     0.00 |  9.15 |     0.323 |
| 5d10         |     0.00 |  8.64 |     0.315 |


In [4]:
print_header()
for num_dice in [1, 2, 3, 4, 5]:
    pool = icepool.d(6).pool(num_dice)
    result = sorting_dice_with_ties.evaluate(pool, pool)
    print_row(f'{num_dice}d6', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 1d6          |     0.00 | 16.67 |     1.095 |
| 2d6          |     0.00 | 20.52 |     0.680 |
| 3d6          |     0.00 | 13.91 |     0.504 |
| 4d6          |     0.00 | 11.71 |     0.404 |
| 5d6          |     0.00 |  9.91 |     0.340 |


In [5]:
# 3.2 Dice Unsorted, With Ties

print_header()
for die_size in [2, 4, 6, 8, 10]:
    die = icepool.d(die_size)
    single = (die - die).sign()
    result = 5 @ single
    print_row(f'5d{die_size}', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d2          |     0.00 | 24.61 |     0.632 |
| 5d4          |     0.00 | 19.32 |     0.516 |
| 5d6          |     0.00 | 16.69 |     0.490 |
| 5d8          |     0.00 | 14.49 |     0.478 |
| 5d10         |     0.00 | 12.71 |     0.471 |


In [6]:
print_header()
for num_dice in [1, 2, 3, 4, 5]:
    single = (icepool.d6 - icepool.d6).sign()
    result = num_dice @ single
    print_row(f'{num_dice}d6', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 1d6          |     0.00 | 16.67 |     1.095 |
| 2d6          |     0.00 | 37.50 |     0.775 |
| 3d6          |     0.00 | 17.82 |     0.632 |
| 4d6          |     0.00 | 23.95 |     0.548 |
| 5d6          |     0.00 | 16.69 |     0.490 |


In [7]:
# 5.1 Rolling Sorted, Player A Wins Ties

class SortingDiceAWinsTies(icepool.OutcomeCountEvaluator):
    def next_state(self, state, outcome, a, b):
        net_score, advantage = state or (0, 0)
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        if advantage < 0:
            net_score -= min(a, -advantage)
        if advantage + a > 0:
            net_score += min(b, advantage + a)
        advantage += a - b
        return net_score, advantage
    
    def final_outcome(self, final_state, *_):
        # Take only the score.
        return final_state[0]
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1
    
sorting_dice_a_wins_ties = SortingDiceAWinsTies()

print_header()
for die_size in [2, 4, 6, 8, 10]:
    pool = icepool.d(die_size).pool(5)
    result = sorting_dice_a_wins_ties.evaluate(pool, pool)
    print_row(f'5d{die_size}', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d2          |    89.06 |  0.00 |     0.238 |
| 5d4          |    54.79 |  0.00 |     0.271 |
| 5d6          |    38.21 |  0.00 |     0.282 |
| 5d8          |    29.13 |  0.00 |     0.287 |
| 5d10         |    23.48 |  0.00 |     0.289 |


In [8]:
# 5.2 Rolling Unsorted, A Wins Ties

print_header()
for die_size in [2, 4, 6, 8, 10]:
    die = icepool.d(die_size)
    single = ((die - die) >= 0).sub({False : -1})
    result = 5 @ single
    print_row(f'5d{die_size}', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d2          |    79.30 |  0.00 |     0.316 |
| 5d4          |    44.96 |  0.00 |     0.400 |
| 5d6          |    30.68 |  0.00 |     0.424 |
| 5d8          |    23.19 |  0.00 |     0.434 |
| 5d10         |    18.63 |  0.00 |     0.439 |


In [9]:
# 6 Reducing Bias With Fewer Dice

print_header()
for num_dice in [2, 3, 4, 5]:
    pool_a = icepool.d(6).pool(num_dice)
    pool_b = icepool.d(6).pool(5)
    result = sorting_dice_a_wins_ties.evaluate(pool_a, pool_b)
    print_row(f'{num_dice}d6 v 5d6', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 2d6 v 5d6    |   -35.61 | 32.37 |     0.608 |
| 3d6 v 5d6    |   -23.63 |  0.00 |     0.451 |
| 4d6 v 5d6    |     3.23 | 20.40 |     0.357 |
| 5d6 v 5d6    |    38.21 |  0.00 |     0.282 |


In [10]:
print_header()
for num_dice in [1, 2, 3, 4]:
    pool_a = icepool.d(6).pool(num_dice)
    pool_b = icepool.d(6).pool(num_dice+1)
    result = sorting_dice_a_wins_ties.evaluate(pool_a, pool_b)
    print_row(f'{num_dice}d6 v {num_dice+1}d6', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 1d6 v 2d6    |   -15.74 |  0.00 |     1.000 |
| 2d6 v 3d6    |    -7.91 | 33.58 |     0.613 |
| 3d6 v 4d6    |    -2.80 |  0.00 |     0.450 |
| 4d6 v 5d6    |     3.23 | 20.40 |     0.357 |


In [11]:
# 7.1 Mixed Dice Sorted, A Wins Ties

print_header()
for num_d8 in range(6):
    pool_a = icepool.d6.pool(5)
    num_d6 = (5 - num_d8)
    die_sizes = (8,) * num_d8 + (6,) * num_d6
    pool_b = icepool.standard_pool(die_sizes)
    result = sorting_dice_a_wins_ties.evaluate(pool_a, pool_b)
    print_row(f'{num_d6}d6/{num_d8}d8', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d6/0d8      |    38.21 |  0.00 |     0.282 |
| 4d6/1d8      |    24.36 |  0.00 |     0.294 |
| 3d6/2d8      |    10.80 |  0.00 |     0.302 |
| 2d6/3d8      |    -2.24 |  0.00 |     0.305 |
| 1d6/4d8      |   -14.56 |  0.00 |     0.305 |
| 0d6/5d8      |   -25.98 |  0.00 |     0.301 |


In [12]:
print_header()
pool_a = icepool.d6.pool(5)

pool_b = icepool.standard_pool([6, 6, 6, 8, 10])
result = sorting_dice_a_wins_ties.evaluate(pool_a, pool_b)
print_row(f'3d6/1d8/1d10', result)

pool_b = icepool.standard_pool([6, 6, 8, 8, 8])
result = sorting_dice_a_wins_ties.evaluate(pool_a, pool_b)
print_row(f'2d6/3d8', result)

pool_b = icepool.standard_pool([6, 6, 6, 10, 10])
result = sorting_dice_a_wins_ties.evaluate(pool_a, pool_b)
print_row(f'3d6/2d10', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 3d6/1d8/1d10 |     2.67 |  0.00 |     0.307 |
| 2d6/3d8      |    -2.24 |  0.00 |     0.305 |
| 3d6/2d10     |    -5.36 |  0.00 |     0.310 |


In [13]:
# 7.2 Mixed Dice Unsorted, A Wins Ties

d6_vs_d6 = ((icepool.d6 - icepool.d6) >= 0).sub({False: -1})
d6_vs_d8 = ((icepool.d6 - icepool.d8) >= 0).sub({False: -1})
d6_vs_d10 = ((icepool.d6 - icepool.d10) >= 0).sub({False: -1})

print_header()
for num_d8 in range(6):
    pool_a = icepool.d6.pool(5)
    num_d6 = (5 - num_d8)
    result = num_d6 @ d6_vs_d6 + num_d8 @ d6_vs_d8
    print_row(f'{num_d6}d6/{num_d8}d8', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d6/0d8      |    30.68 |  0.00 |     0.424 |
| 4d6/1d8      |    20.34 |  0.00 |     0.440 |
| 3d6/2d8      |     9.48 |  0.00 |     0.450 |
| 2d6/3d8      |    -1.61 |  0.00 |     0.452 |
| 1d6/4d8      |   -12.60 |  0.00 |     0.446 |
| 0d6/5d8      |   -23.19 |  0.00 |     0.434 |


In [14]:
print_header()
result = 3 @ d6_vs_d6 + 2 @ d6_vs_d8
print_row('3d6/2d8', result)

result = 3 @ d6_vs_d6 + 1 @ d6_vs_d8 + 1 @ d6_vs_d10
print_row('3d6/1d8/1d10', result)

result = 2 @ d6_vs_d6 + 3 @ d6_vs_d8
print_row('2d6/3d8', result)

result = 3 @ d6_vs_d6 + 2 @ d6_vs_d10
print_row('3d6/2d10', result)

result = 2 @ d6_vs_d6 + 2 @ d6_vs_d8 + 1 @ d6_vs_d10
print_row('2d6/2d8/1d10', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 3d6/2d8      |     9.48 |  0.00 |     0.450 |
| 3d6/1d8/1d10 |     2.97 |  0.00 |     0.456 |
| 2d6/3d8      |    -1.61 |  0.00 |     0.452 |
| 3d6/2d10     |    -3.73 |  0.00 |     0.459 |
| 2d6/2d8/1d10 |    -8.26 |  0.00 |     0.453 |


In [15]:
# 8.1 Rolling Sorted, Rerolling Tied Dice

class SortingDiceCountTies(icepool.OutcomeCountEvaluator):
    def next_state(self, state, outcome, a, b):
        net_score, ties, advantage = state or (0, 0, 0)
        # Advantage is the number of unpaired dice that rolled a previous (higher) number.
        # If positive, it favors side A, otherwise it favors side B.
        # We pair them off with newly-rolled dice of the disadvantaged side.
        if advantage > 0:
            abs_delta = min(b, advantage)
            net_score += abs_delta
            ties += min(a, b - abs_delta)
        elif advantage < 0:
            abs_delta = min(a, -advantage)
            net_score -= abs_delta
            ties += min(b, a - abs_delta)
        else:
            ties += min(a, b)
        advantage += a - b
        return net_score, ties, advantage
    
    def final_outcome(self, final_state, *pools):
        net_score, ties, advantage = final_state
        # Reroll if every pair was tied.
        # This ensures that we make progress towards termination every sub-game.
        if ties == min(pool.size() for pool in pools):
            return icepool.Reroll
        return net_score, ties
    
    def direction(self, *_):
        # See outcomes in descending order.
        return -1

sorting_dice_count_ties = SortingDiceCountTies()

print_header()
for die_size in [2, 4, 6, 8, 10]:
    die = icepool.d(die_size)
    def sub_game(state):
        net_score, ties = state
        if ties == 0:
            return net_score, 0
        else:
            pool = die.pool(ties)
            # Die addition is element-wise on tuples.
            return sorting_dice_count_ties(pool, pool) + (net_score, 0)
    initial = (0, 5)  # zero starting score, 5 dice
    result = icepool.Die([initial]).sub(sub_game, max_depth=5).marginals[0]
    print_row(f'5d{die_size}', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d2          |     0.00 |  0.00 |     0.350 |
| 5d4          |     0.00 |  0.00 |     0.311 |
| 5d6          |     0.00 |  0.00 |     0.302 |
| 5d8          |     0.00 |  0.00 |     0.298 |
| 5d10         |     0.00 |  0.00 |     0.296 |


In [16]:
print_header()
for num_dice in [1, 2, 3, 4, 5]:
    def sub_game(state):
        net_score, ties = state
        if ties == 0:
            return net_score, 0
        else:
            pool = icepool.d6.pool(ties)
            # Die addition is element-wise on tuples.
            return sorting_dice_count_ties(pool, pool) + (net_score, 0)
    initial = (0, num_dice)
    result = icepool.Die([initial]).sub(sub_game, max_depth=num_dice).marginals[0]
    print_row(f'{num_dice}d6', result)
    if num_dice == 2:
        two_d6_tie_percentage = tie_percentage(result)

# The paper (which uses Monte Carlo for this table)
# has a different value for the Tie % in the last decimal place.
# However, with the paper using 6^10 samples,
# the rounding threshold is within 1.42 standard deviations of our calculated value.
# So the difference is plausibly explained by random noise.
print(f'2d6 tie percentage: {two_d6_tie_percentage:0.4f}')
sd = sqrt(two_d6_tie_percentage * (100.0 - two_d6_tie_percentage) / 6**10)
print(f'Standard deviation of percentage: {sd:0.4f}')
error_sd = (34.155 - two_d6_tie_percentage) / sd
print(f'Error in standard deviations: {error_sd:0.4f}')

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 1d6          |     0.00 |  0.00 |     1.000 |
| 2d6          |     0.00 | 34.15 |     0.616 |
| 3d6          |     0.00 |  0.00 |     0.453 |
| 4d6          |     0.00 | 21.00 |     0.361 |
| 5d6          |     0.00 |  0.00 |     0.302 |
2d6 tie percentage: 34.1463
Standard deviation of percentage: 0.0061
Error in standard deviations: 1.4198


In [17]:
# 8.2 Rolling Unsorted, Rerolling Ties

print_header()
for die_size in [2, 4, 6, 8, 10]:
    die = icepool.d(die_size)
    single = (die - die).sign().reroll({0})
    result = 5 @ single
    print_row(f'5d{die_size}', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 5d2          |     0.00 |  0.00 |     0.447 |
| 5d4          |     0.00 |  0.00 |     0.447 |
| 5d6          |     0.00 |  0.00 |     0.447 |
| 5d8          |     0.00 |  0.00 |     0.447 |
| 5d10         |     0.00 |  0.00 |     0.447 |


In [18]:
single = (icepool.d6 - icepool.d6).sign().reroll({0})

print_header()
for num_dice in [1, 2, 3, 4, 5]:
    result = num_dice @ single
    print_row(f'{num_dice}d6', result)

|     Game     | win bias | tie % | closeness |
|:------------:|---------:|------:|----------:|
| 1d6          |     0.00 |  0.00 |     1.000 |
| 2d6          |     0.00 | 50.00 |     0.707 |
| 3d6          |     0.00 |  0.00 |     0.577 |
| 4d6          |     0.00 | 37.50 |     0.500 |
| 5d6          |     0.00 |  0.00 |     0.447 |


In [19]:
end_ns = time.perf_counter_ns()
elapsed_s = (end_ns - start_ns) * 1e-9
print(f'Elapsed time after loading: {elapsed_s:0.3f} s')

Elapsed time after loading: 1.514 s
